# Rendu de projet données immobilières

Useful links : [Subject](https://simplonline.co/briefs/327207de-b68d-4563-bc7a-249a7637782d) | [Original dataset (.csv)](https://www.data.gouv.fr/fr/datasets/5c4ae55a634f4117716d5656/) <br>
This project was realized by : Adil, Marianne, Nidal, Théo & Zohra

----
## 📍 TO DO :
- Find a solution for data type of col `Code postal`
- Last question (n. 12)
- Add links to md
- Modify data dict accordingly
- Does we need to present data in formal way ?
<br>
<br>
-----

### Table of content :
> 💡 Links are working if you clik them in theo's GitHub repo
- [Dictionnaire de données - Data Dictionary]()
- [Modèle Conceptuel de données (MCD)]()
- [Modèle Physique de données (MPD)]()

### Table of questions :
1. [Nombre d'appartements et maisons vendus en 2020](#) 
2. [Nombre de biens vendu par trimestre](#)
3. [Proportion des ventes de biens par trimestre par typologie de bien](#)
4. [Proportion d’appartements vendus par nombre de pièces](#)
5. [Les 10 départements où il y a eu le plus de ventes immobilières](#)
6. [Les 10 départements où il y en a eu le moins](#)
7. [Prix au m2 en IDF](#)
8. [Liste des 10 appartements les plus chers avec le département et le nombre de mètres carrés](#)
9. [Taux d’évolution du nombre de ventes entre le premier et le second trimestre de 2020](#)
10. [Liste des communes où le nombre de ventes a augmenté d'au moins 20% entre le premier et le second trimestre de 2020](#)

In [217]:
# PG connection
import psycopg2 as pgconn

def connect():
    try:
        conn = psycopg2.connect(
            host='localhost',
            database='immo_db',
            user='postgres',
            password=''
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print('⚠︎ Postgres connection error')
        print(error)
    finally:
        print('Connection established with postgres.')
        return conn
    
conn = connect()
conn.autocommit = True
cur = conn.cursor()

# Import pandas
import pandas as pd

# Import numpy
import numpy as np

# Import seaborn
import seaborn as sn

Connection established with postgres.


----------
## Nombre d'appartements et maisons vendus en 2020

```sql
SELECT sum(total_houses + total_flats) AS sub_total, total_houses, total_flats FROM (
	SELECT * FROM 
	(
		SELECT sum(case when bien.code_type_local = 1 then 1 else 0 end) AS total_houses, 
		sum(case when bien.code_type_local = 2 then 1 else 0 end) AS total_flats
		FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id 
	) x 
) y GROUP BY total_flats, total_houses;
```

In [220]:
cur.execute('SELECT sum(total_houses + total_flats) AS sub_total, total_houses, total_flats FROM (SELECT * FROM (SELECT sum(case when bien.code_type_local = 1 then 1 else 0 end) AS total_houses, sum(case when bien.code_type_local = 2 then 1 else 0 end) AS total_flats FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id ) x ) y GROUP BY total_flats, total_houses;')
result = cur.fetchone()

df = pd.DataFrame(result)
df = df.transpose()
df = df.rename(columns={0: 'Total', 1: 'Sub-total houses', 2: 'Sub-total flats'})

df

,Total,Sub-total houses,Sub-total flats
0,299,217,82


------------
##  Nombre de biens vendu par trimestre 

```sql
SELECT 
	sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
	sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2,
	sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3,
	sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4
FROM transaction;
```

In [211]:
cur.execute("SELECT sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1, sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2, sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3, sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4 FROM transaction")
result = cur.fetchone()

df = pd.DataFrame(result)
df = df.transpose()
df = df.rename(columns={0: 'T1', 1: 'T2', 2: 'T3', 3: 'T4'}, index={0 : '2020'})
df

,T1,T2,T3,T4
2020,1000,0,0,0


-----------
##  Proportion des ventes de biens par trimestre par typologie de bien

```sql
SELECT 
	sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
	sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2,
	sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3,
	sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4
FROM transaction LEFT JOIN bien on transaction.bien_id = bien.id GROUP BY bien.code_type_local;
```

In [215]:
cur.execute("SELECT sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1, sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2, sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3, sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4 FROM transaction LEFT JOIN bien on transaction.bien_id = bien.id GROUP BY bien.code_type_local;")
result = cur.fetchall()

df = pd.DataFrame(result, columns=['T1', 'T2', 'T3', 'T4'], index=['Houses', 'Flats', 'Isolated dependancy', 'Industrial space', 'Not specified'])
df

,T1,T2,T3,T4
Houses,602,0,0,0
Flats,217,0,0,0
Isolated dependancy,76,0,0,0
Industrial space,23,0,0,0
Not specified,82,0,0,0


--------------
##  Proportion d’appartements vendus par nombre de pièces

```sql
SELECT count(transaction.id), bien.nb_pieces_principales FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id WHERE bien.code_type_local = 2 GROUP BY bien.nb_pieces_principales ORDER BY bien.nb_pieces_principales;
```

In [144]:
cur.execute('SELECT count(transaction.id), bien.nb_pieces_principales FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id WHERE bien.code_type_local = 2 GROUP BY bien.nb_pieces_principales ORDER BY bien.nb_pieces_principales;')
result = cur.fetchall()

df = pd.DataFrame(result, columns=['Total nb of flats', 'Nb of rooms'])
df

,Total nb of flats,Nb of rooms
0,6,1
1,23,2
2,29,3
3,17,4
4,7,5


------------
## Les 10 départements où il y a eu le plus de ventes immobilières

❌ ERR : Attention, trouver une solution pour ne pas perdre le CP (cast en float auto de `pd.read_csv()` dénature la colonne `Code postal` : ie. les 9 premiers départements disparaissent) 

```sql
SELECT count(transaction.id), substr(code_postal, 0, 3) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id GROUP BY substr(code_postal, 0, 3) ORDER BY count(transaction.id) DESC LIMIT 10
```

------------
##  Les 10 départements où il y en a eu le moins

```sql
SELECT count(transaction.id), substr(code_postal, 0, 3) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id GROUP BY substr(code_postal, 0, 3) ORDER BY count(transaction.id) ASC LIMIT 10
```

--------------
##  Prix moyen du mètre carré en IDF

```sql
	SELECT (sum(transaction.valeur_fonciere) / sum(bien.surface_reel_bati)) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id WHERE substr(bien.code_postal, 0, 3) IN ('75', '92', '93', '94')
```

In [167]:
cur.execute("SELECT (sum(transaction.valeur_fonciere) / sum(bien.surface_reel_bati)) FROM transaction LEFT JOIN bien ON transaction.bien_id = bien.id WHERE substr(bien.code_postal, 0, 3) IN ('75', '92', '93', '94');")
result = cur.fetchone()

print(f'The medium price in IDF is {result[0] or False}/m2')

The medium price in IDF is False/m2


------------------
##  Liste des 10 appartements les plus chers avec le département et le nombre de mètres carrés

```sql
SELECT * FROM (
	SELECT valeur_fonciere, code_postal, surface_reel_bati FROM transaction LEFT JOIN bien on transaction.bien_id = bien.id ORDER BY transaction.valeur_fonciere DESC
) x WHERE (-1 IN (valeur_fonciere)) IS NOT NULL LIMIT 10
```

In [156]:
cur.execute('SELECT * FROM (SELECT valeur_fonciere, code_postal, surface_reel_bati FROM transaction LEFT JOIN bien on transaction.bien_id = bien.id ORDER BY transaction.valeur_fonciere DESC) x WHERE (-1 IN (valeur_fonciere)) IS NOT NULL LIMIT 10;')
result = cur.fetchall()

df = pd.DataFrame(result, columns=['Land value', 'Postal code', 'Surface'])
df

,Land value,Postal code,Surface
0,1250000.0,1000,558.0
1,1250000.0,1000,554.0
2,1250000.0,1000,NaN
3,715350.0,1851,NaN
4,715350.0,1851,400.0
5,715350.0,1851,NaN
6,715350.0,1851,NaN
7,480000.0,1240,NaN
8,480000.0,1240,NaN
9,480000.0,1240,1408.0


--------------
## Taux d’évolution du nombre de ventes entre le premier et le second trimestre de 2020

Evolution rate formula : $ {\displaystyle t={\frac {V_{A}-V_{D}}{|V_{D}|}}} $ where t is expressed in percentage (%)



```sql
SELECT (t2 - t1)::FLOAT / t1 FROM
	(SELECT 
		sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1,
		sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2
	FROM transaction) AS ts
```

In [145]:
cur.execute("SELECT (t2 - t1)::FLOAT / t1 FROM (SELECT sum(case when date_mutation <= '2020-03-31' then 1 else 0 end) as t1, sum(case when date_mutation BETWEEN '2020-04-01' AND '2020-06-30' then 1 else 0 end) as t2, sum(case when date_mutation BETWEEN '2020-07-01' AND '2020-09-30' then 1 else 0 end) as t3, sum(case when date_mutation >= '2020-10-01' then 1 else 0 end) as t4 FROM transaction) AS ts")
result = cur.fetchone()

print(f'The evolution rate bewteen the first and second semester of 2020 is equal to {round(result[0], 3)}%')


The evolution rate bewteen the first and second semester of 2020 is equal to -1.0%


-------------
##  Liste des communes où le nombre de ventes a augmenté d'au moins 20% entre le premier et le second trimestre de 2020

In [218]:
# TO DO